In [1]:
import os
from os import path
import pandas as pd
import sys
basepath = path.join(os.getcwd(), "export_us")

In [2]:
import os
import numpy as np
import pandas as pd

start, end = '2015-06-30', '2024-06-27'
stock_codes = pd.read_csv('dji_companies.csv')['Symbol']

dfs = []
for stock_code in stock_codes:
    fname = f"{stock_code}.csv"
    fpath = os.path.join(basepath, fname)
    if os.path.exists(fpath):
        df = pd.read_csv(fpath, parse_dates=['日期'])
        df.rename(columns={'日期': 'date', '开盘': 'open', '收盘': 'close', '最高': 'high', '最低': 'low', '成交量': 'volume', '成交额': 'amount'}, inplace=True)
        df['stock_code'] = stock_code
        dfs.append(df[(df['date'] >= start) & (df['date'] <= end)])

all_df = pd.concat(dfs, ignore_index=True)
all_df.set_index(['stock_code', 'date'], inplace=True)
all_df.sort_index(inplace=True)

In [3]:
all_df

open    close     high      low     volume  \
stock_code date                                                        
AAPL       2015-06-30   23.203   23.165   23.340   23.025   43794410   
           2015-07-01   23.535   23.440   23.545   23.308   30028055   
           2015-07-02   23.418   23.410   23.483   23.253   26997705   
           2015-07-06   23.045   23.310   23.368   23.023   27844079   
           2015-07-07   23.283   23.233   23.348   22.753   46692274   
...                        ...      ...      ...      ...        ...   
NVDA       2024-06-21  127.070  126.520  130.580  124.250  655484672   
           2024-06-24  123.190  118.060  124.410  117.990  476060864   
           2024-06-25  121.150  126.040  126.450  119.270  425787456   
           2024-06-26  126.080  126.350  128.070  122.550  362975920   
           2024-06-27  124.050  123.940  126.360  122.870  252571693   

                             amount    振幅   涨跌幅    涨跌额   换手率  
stock_code date                                               
AAPL       2015-06-30  0.000000e+00  1.37  0.95  0.217  0.77  
           2015-07-01  0.000000e+00  1.02  1.19  0.275  0.53  
           2015-07-02  0.000000e+00  0.98 -0.13 -0.030  0.47  
           2015-07-06  0.000000e+00  1.47 -0.43 -0.100  0.49  
           2015-07-07  0.000000e+00  2.55 -0.33 -0.077  0.82  
...                             ...   ...   ...    ...   ...  
NVDA       2024-06-21  8.324440e+10  4.84 -3.22 -4.210  2.66  
           2024-06-24  5.732459e+10  5.07 -6.69 -8.460  1.94  
           2024-06-25  5.266794e+10  6.08  6.76  7.980  1.73  
           2024-06-26  4.534285e+10  4.38  0.25  0.310  1.48  
           2024-06-27  3.138866e+10  2.76 -1.91 -2.410  1.03  

[15848 rows x 10 columns]

In [4]:
all_df

open    close     high      low     volume  \
stock_code date                                                        
AAPL       2015-06-30   23.203   23.165   23.340   23.025   43794410   
           2015-07-01   23.535   23.440   23.545   23.308   30028055   
           2015-07-02   23.418   23.410   23.483   23.253   26997705   
           2015-07-06   23.045   23.310   23.368   23.023   27844079   
           2015-07-07   23.283   23.233   23.348   22.753   46692274   
...                        ...      ...      ...      ...        ...   
NVDA       2024-06-21  127.070  126.520  130.580  124.250  655484672   
           2024-06-24  123.190  118.060  124.410  117.990  476060864   
           2024-06-25  121.150  126.040  126.450  119.270  425787456   
           2024-06-26  126.080  126.350  128.070  122.550  362975920   
           2024-06-27  124.050  123.940  126.360  122.870  252571693   

                             amount    振幅   涨跌幅    涨跌额   换手率  
stock_code date                                               
AAPL       2015-06-30  0.000000e+00  1.37  0.95  0.217  0.77  
           2015-07-01  0.000000e+00  1.02  1.19  0.275  0.53  
           2015-07-02  0.000000e+00  0.98 -0.13 -0.030  0.47  
           2015-07-06  0.000000e+00  1.47 -0.43 -0.100  0.49  
           2015-07-07  0.000000e+00  2.55 -0.33 -0.077  0.82  
...                             ...   ...   ...    ...   ...  
NVDA       2024-06-21  8.324440e+10  4.84 -3.22 -4.210  2.66  
           2024-06-24  5.732459e+10  5.07 -6.69 -8.460  1.94  
           2024-06-25  5.266794e+10  6.08  6.76  7.980  1.73  
           2024-06-26  4.534285e+10  4.38  0.25  0.310  1.48  
           2024-06-27  3.138866e+10  2.76 -1.91 -2.410  1.03  

[15848 rows x 10 columns]

In [5]:
all_df.to_csv("all_stock_data.csv")

In [6]:
all_df_loaded = pd.read_csv("all_stock_data.csv", parse_dates=['date'])
all_df_loaded.set_index(['stock_code', 'date'], inplace=True)
all_df_loaded.sort_index(inplace=True)

In [7]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import pearsonr


# 读取合并后的数据
all_df_loaded = pd.read_csv("all_stock_data.csv", parse_dates=['date'])
all_df_loaded.set_index(['stock_code', 'date'], inplace=True)
all_df_loaded.sort_index(inplace=True)


# 定义你的因子函数，输入为单个股票的DataFrame
def factor_func(df):
    # Calculate the difference between close and open prices, representing the price change over a day
    price_change = df['close'] - df['open']
    
    # Calculate typical price
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    
    # Compute the ratio of volume to typical price as a measure of market interest
    volume_interest = df['volume'] / typical_price
    
    # Calculate the 10-day moving average of the typical price
    ma_10 = typical_price.rolling(window=10).mean()
    
    # Calculate the 5-day exponential moving average of the typical price
    ema_5 = typical_price.ewm(span=5, adjust=False).mean()
    
    # Calculate the Volume-Weighted Average Price (VWAP)
    vwap = (df['close'] * df['volume']).cumsum() / df['volume'].cumsum()
    
    # Adaptive window for volatility: calculate the standard deviation over an adaptive window
    window_size = 20
    volatility = typical_price.rolling(window=window_size).std()
    
    # Calculate the 14-day RSI
    delta = df['close'].diff(1)
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    # Calculate the 20-day momentum
    momentum_20 = df['close'] - df['close'].shift(20)
    
    # Calculate the 50-day and 200-day moving averages of the close price
    ma_50 = df['close'].rolling(window=50).mean()
    ma_200 = df['close'].rolling(window=200).mean()
    
    # Calculate the 50-day and 200-day Bollinger Bands
    bb_50_middle = df['close'].rolling(window=50).mean()
    bb_50_std = df['close'].rolling(window=50).std()
    bb_50_upper = bb_50_middle + 2 * bb_50_std
    bb_50_lower = bb_50_middle - 2 * bb_50_std
    
    bb_200_middle = df['close'].rolling(window=200).mean()
    bb_200_std = df['close'].rolling(window=200).std()
    bb_200_upper = bb_200_middle + 2 * bb_200_std
    bb_200_lower = bb_200_middle - 2 * bb_200_std
    
    # Calculate the percentage of the close price relative to the 50-day and 200-day Bollinger Bands
    bb_50_percent = (df['close'] - bb_50_lower) / (bb_50_upper - bb_50_lower)
    bb_200_percent = (df['close'] - bb_200_lower) / (bb_200_upper - bb_200_lower)
    
    # Adaptive weights based on recent data and market conditions
    momentum_weight = 0.1 + 0.05 * (rsi - 50)
    mean_reversion_weight = 0.1 + 0.05 * (1 - (typical_price - ma_10) / volatility)
    trend_weight = 0.1 + 0.05 * ((ma_50 - ma_200) / volatility)
    
    # Create a weighted indicator using the price change, volume interest, and additional technical indicators
    alpha_factor = (
        0.4 * price_change 
        + 0.2 * volume_interest 
        + 0.1 * (ema_5 - ma_10) 
        + 0.1 * (vwap - typical_price) / volatility 
        + 0.1 * (rsi - 50) 
        + momentum_weight * momentum_20 
        + mean_reversion_weight * (ma_50 - ma_200) 
        + trend_weight * (bb_50_percent - bb_200_percent)
    )
    
    return alpha_factor

# 计算每只股票的因子值
all_df_loaded['factor'] = all_df_loaded.groupby('stock_code').apply(lambda g: factor_func(g)).reset_index(level=0, drop=True)

# 计算未来6日收益率
all_df_loaded['future_return_6d'] = all_df_loaded.groupby('stock_code')['close'].shift(-6) / all_df_loaded['close'] - 1

# 取所有日期
start_date = pd.Timestamp('2021-01-01')
end_date = pd.Timestamp('2024-01-01')
all_dates = all_df_loaded.index.get_level_values('date').unique()
all_dates = all_dates[(all_dates >= start_date) & (all_dates <= end_date)]
ic_values = []

for date in all_dates:
    daily = all_df_loaded.xs(date, level='date')
    factors = daily['factor']
    returns = daily['future_return_6d']
    mask = factors.notna() & returns.notna() & np.isfinite(factors) & np.isfinite(returns)
    if mask.sum() >= 10:
        ic, _ = pearsonr(factors[mask], returns[mask])
        if not np.isnan(ic):
            ic_values.append(ic)

mean_ic = np.mean(ic_values)
print(f"Mean IC: {mean_ic:.10f}")

Mean IC: nan


/home/rjj/miniconda3/envs/StockPred/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/rjj/miniconda3/envs/StockPred/lib/python3.11/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
